### 10 Emoji Prediction

The task of this project is to make a system that would automatically fill the text with the appropriate emoticons. This can be done in two steps. First, for each position within the text a prediction is made whether an emoticon should be placed there. Second, an appropriate emoticon is chosen from a list of available emoticons. Both these tasks can be set up as supervised classification problems.

Competition website:
https://competitions.codalab.org/competitions/17344

Dataset:
https://competitions.codalab.org/competitions/17344

Entry point:
https://arxiv.org/pdf/1702.07285.pdf (Barbieri, Francesco, Miguel Ballesteros, and Horacio Saggion. Are Emojis Predictable?)

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Define paths to folders containing the data and the results.


In [2]:
NUMBER_OF_TWEETS = None
# NUMBER_OF_TWEETS = 100000

MAX_WORDS_PER_TWEET = 30
# DATA_LOCATION = "./train/data/noise/"
DATA_LOCATION = "./train/data/"
# DATA_LOCATION = "./train/data/words/"
RESULT_LOCATION = "./results/"
#TWEET_FILE_NAME = "tweet_by_ID_28_4_2018__03_20_05" + "_"
TWEET_FILE_NAME = "tweet_by_ID_02_6_2018__05_40_21__no_hashtags" + "_"

k = 3

if NUMBER_OF_TWEETS is not None:
    TWEET_FILE_NAME += str(NUMBER_OF_TWEETS)
else:
    TWEET_FILE_NAME += "ALL"

### Load the data
Tweets are loaded in two ways: list of strings (for the TF-IDF vectorizer) and a list of lists of words (for feature extraction). Labels are read as a numpy array of N * MAX_WORDS_PER_TWEET dimensions.

In [3]:
base_file_name = DATA_LOCATION + TWEET_FILE_NAME

text_lines = []
text_lines_split = []

with open(base_file_name + ".text", 'r', encoding="utf-8") as out_text:
    for line in out_text:
        text_lines.append(line[:-1])
        text_lines_split.append(line[:-1].split())
        
loc_lines = []
with open(base_file_name + ".loclabels", 'r') as loc_labels:
    for line in loc_labels:
        loc_line = []
        for c in line[:-1]:
            loc_line.append(int(c))
        loc_lines.append(loc_line)

loc_lines = np.asarray(loc_lines)

emo_lines = []
with open(base_file_name + ".emolabels", 'r') as emo_labels:
    for e_line, loc in zip(emo_labels, loc_lines):
        emo_line = [0]*31
        e_line2 = e_line.split()
        
        br = 0
        for idx, val in enumerate(loc[:-1]):
            if val==1:
                emo_line[idx]=int(e_line2[br])+1
                br += 1
        emo_lines.append(emo_line)
        
emo_lines = np.asarray(emo_lines)

print(f"number of tweets {len(text_lines)}")
print(f"example of tweet texts:")
for i in range(10):
    print(f"'{text_lines[i]}'")
print(f"\nexample of labels (emoji locations):\n{loc_lines[:10]}")
print(f"\nexample of labels (emoji type):\n{emo_lines[:10]}")

number of tweets 473459
example of tweet texts:
'lol @ west covina , california'
'things got a little festive at the office @ redrock'
'step out and explore . @ ellis island cafe'
'@user @ cathedral preparatory school'
'my baby bear @ bubby's'
'rupaul's drag race bingo fun . drag queens be sexy ! @user abwyman'
'black history like a mufffffaaaaaka done thru her yugioh trap card like hell'
'just light makeup'
'@ bj's restaurant and brewhouse'
'so lovely catching up with my soul sister @user @ university of victoria'

example of labels (emoji locations):
[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

### TF-IDF Feature Extraction
TF-IDF is computed on the collection of tweets. Then for every position between words a new example is generated: a 2 * k array containing the k left and k right tfidf values of words. 
Labels for locations are taken as 1 or 0 wether an emoji was there in the original tweet.
Labels for emojis are mapped to a number between 0 and 20 where 20 marks no appearance of emoji and the rest describes 20 most frequent emojis.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random

X_tfidf = None
word_to_tfidf_index_dict = {}

def tfidf_features(tweets, loc_labels, emo_labels, k, func):
    global word_to_tfidf_index_dict, X_tfidf
    
    tfidf_model = TfidfVectorizer(input="content", analyzer="word", stop_words="english")
    X_tfidf = tfidf_model.fit_transform(text_lines)

    #word_to_tfidf_index_dict = {}
    for i, word in enumerate(tfidf_model.get_feature_names()):
        word_to_tfidf_index_dict[word] = i

    print(f"X shape {X_tfidf.shape}")
    print(f"y_l shape {loc_lines.shape}")
    print(f"y_e shape {emo_lines.shape}")
    print(f"some tf-idf values\n{X_tfidf[0]}\n")

    N = len(tweets)
    X = []
    y_l = []
    y_e = []
    
    for tweet_index, (tweet, l_label, e_label) in enumerate(zip(tweets, loc_labels, emo_labels)):
        for pos in range(len(tweet) + 1):
                
            x = []
            for i in range(pos - k, pos + k):
                if i < 0 or i >= len(tweet):
                    x.append(0.0)
                else:
                    x.append(func(tweet_index, tweet[i]))
            X.append(x)
            y_l.append(l_label[pos])
            y_e.append(e_label[pos])
            
    return np.asarray(X), np.asarray(y_l), np.asarray(y_e)

def word_to_tfidf(tweet_index, word):
    global word_to_tfidf_index_dict, X_tfidf
    
    if word in word_to_tfidf_index_dict:
        return X_tfidf[tweet_index, word_to_tfidf_index_dict[word]]
    else:
        return 0.0


def print_dataset(X, y_l, y_e):
    emoji_num = np.count_nonzero(y_l)
    class_freq_ratio = emoji_num / (X.shape[0] * X.shape[1])
    
    print("after feature extraction:")
    print(f"X shape {X.shape}")
    print(f"y_l shape {y_l.shape}")
    print(f"y_e shape {y_e.shape}")
    print(f"some X values\n{X[:5]}")
    print(f"some y_l values\n{y_l[:5]}")
    print(f"non zero elements (1 in label) in y_l {emoji_num}")
    print(f"class frequency ratio {class_freq_ratio}\n")
    print(f"some y_e values\n{y_e[:5]}")

### Word2Vector Feature Extraction

A similar dataset is obtained using w2vec. By setting the number of features to $2*k=6$, the dimensionality stays the same as the previous tf-idf feature extraction. For each word a 6 dimensional vector is obtained and then the average of the 6 neighboring words is taken. Labels stay the same.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

num_of_features = 2 * k
w2v_model_file_name = "./models/w2v_model"

In [6]:
w2v_model = Word2Vec(text_lines_split, min_count=1, size=num_of_features)
w2v_model.save(w2v_model_file_name)

In [7]:
w2v_model = Word2Vec.load(w2v_model_file_name)

example = w2v_model.wv["california"]
print(example.shape)
print(example)

(6,)
[ 8.471104    4.5863085   0.05026486 -2.7232955  -4.0103526   6.726843  ]


In [8]:
def w2v_features(tweets, labels, k):
    N = len(tweets)
    X = []
    y = []
    
    for tweet_index, (tweet, label) in enumerate(zip(tweets, labels)):
        for pos in range(len(tweet) + 1):
            x = []
            for i in range(pos - k, pos + k):
                if i < 0 or i >= len(tweet):
                    x.append(np.zeros(num_of_features))
                else:
                    x.append(w2v_model.wv[tweet[i]])
            x = np.average(x, axis=0)
            X.append(x)
            y.append(label[pos])
            
    return np.asarray(X), np.asarray(y)

In [9]:
from sklearn.model_selection import train_test_split

# should_use_tfidf = True
# should_use_hybrid = False

# X_tfidf, y_tfidf = tfidf_features(text_lines_split, loc_lines, k, word_to_tfidf)
# X_w2v, y_w2v = w2v_features(text_lines_split, loc_lines, k)
# X_hybrid = np.hstack((X_tfidf, X_w2v))

# if should_use_tfidf or should_use_hybrid:    
#     X, y = X_tfidf, y_tfidf

# if (not should_use_tfidf) or should_use_hybrid:
#     X, y = X_w2v, y_w2v

# if should_use_hybrid:
#     X = X_hybrid

# print_dataset(X, y)

X_tfidf, y_loc_tfidf, y_emo_tfidf = tfidf_features(text_lines_split, loc_lines, emo_lines, k, word_to_tfidf)

X_train, X_test, y_loc_train, y_loc_test = train_test_split(X_tfidf, y_loc_tfidf, shuffle=False, test_size=0.1, random_state=42)
X_train, X_test, y_emo_train, y_emo_test = train_test_split(X_tfidf, y_emo_tfidf, shuffle=False, test_size=0.1, random_state=42)

/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


X shape (473459, 134815)
y_l shape (473459, 31)
y_e shape (473459, 31)
some tf-idf values
  (0, 73855)	0.41212570578147684
  (0, 129964)	0.4318968334528044
  (0, 31497)	0.7178895542721762
  (0, 23363)	0.35812304555062646



### Baselines

Make sure to check should_train flags when training/testing.

In [10]:
should_use_kfold = True

should_train_global = True
should_train_linear_svm = True
should_train_bagging_svm = False
should_train_random_forest = False
should_train_adaboost = True

baseline_models_dir = "./models/baseline_models/"
linear_svm_model_file_name = baseline_models_dir + "linear_svm.pkl"
bagging_model_file_name = baseline_models_dir + "bagging_svm.pkl"
random_forest_file_name = baseline_models_dir + "random_forest.pkl"
adaboost_file_name = baseline_models_dir + "adaboost.pkl"

In [11]:
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, KFold

np.set_printoptions(threshold=np.inf)
EVALUATION_RESULTS_KEYS = ['Accuracy: ', 'Precision: ', 'Recall: ', 'F1: ', 'ratio of positive/negative predictions: ']

kf = KFold(n_splits=5, shuffle=True)


def calc_scores(y_pred, y_test, multi_class):
    result = []
    
    if multi_class:
        result_temp = []
    
        for i in range(0, 21):
            vfunc = np.vectorize(vectorize_func)
            y_test_new = vfunc(y_test, i)
            y_pred_new = vfunc(y_pred, i)
            
            result_temp2 = []
            alfa=1
            if i == 0:
                alfa=1/5
            
            result_temp2.append(alfa*accuracy_score(y_pred_new, y_test_new))
            result_temp2.append(alfa*precision_score(y_pred_new, y_test_new))
            result_temp2.append(alfa*recall_score(y_pred_new, y_test_new))
            result_temp2.append(alfa*f1_score(y_pred_new, y_test_new))
            result_temp.append(result_temp2)
    
        result_avg = np.average(np.asarray(result_temp), axis=0)
        result = result_avg[:]
    else:
        result.append(accuracy_score(y_pred, y_test))
        result.append(precision_score(y_pred, y_test))
        result.append(recall_score(y_pred, y_test))
        result.append(f1_score(y_pred, y_test))
        result.append(np.count_nonzero(y_pred) / y_pred.shape[0])
    return result

def print_scores(scores):
    print(f"Accuracy: {scores[0]}")
    print(f"Precision: {scores[1]}")
    print(f"Recall: {scores[2]}")
    print(f"F1: {scores[3]}")
    if len(scores) > 4:
        print(f"ratio of positive/negative predictions {scores[4]}")

def kfold_score(clf, X, y, multi_class, should_print=True):
    fold_scores = []
    for train_index, test_index in kf.split(X):
        X_train_kf, X_test_kf = X[train_index], X[test_index]
        y_train_kf, y_test_kf = y[train_index], y[test_index]
        clf.fit(X_train_kf, y_train_kf)
        y_pred = clf.predict(X_test_kf)
        fold_scores.append(calc_scores(y_pred, y_test_kf, multi_class))
        
    fold_scores = np.average(np.asarray(fold_scores), axis=0)
    if should_print:
        print_scores(fold_scores)
    return fold_scores

def train_and_save_model(model, X_train, y_train, model_file_name):
    model.fit(X_train, y_train)
    joblib.dump(model, model_file_name)
    
def load_and_test_model(model_file_name, X_test, y_test):
    clf_loaded = joblib.load(model_file_name)
    y_pred = clf_loaded.predict(X_test)
    print_scores(calc_scores(y_pred, y_test))
    
def do_test(clf, X, y, clf_file_name, flag, multi_class=False):
    if should_use_kfold:
        return kfold_score(clf, X, y, multi_class)
    else:
        if should_train_global and flag:
            train_and_save_model(clf, X_train, y_train, clf_file_name)
        return load_and_test_model(clf_file_name, X_test, y_test, multi_class) 
        
def write_to_file(best_params, results, baseline_name, loc_or_emo):
    timestr = time.strftime("%d_%m_%Y__%H_%M_%S_")
    FOLDER_LOCATION = RESULT_LOCATION + "baselines/" + baseline_name + "/"
    FILE_NAME = baseline_name + "_" + loc_or_emo + "_" + timestr
    if NUMBER_OF_TWEETS is not None:
        FILE_NAME += str(NUMBER_OF_TWEETS)
    else:
        FILE_NAME += "ALL"
    FILE_NAME += '.text'
    RESULTS_FILE_NAME = FOLDER_LOCATION + FILE_NAME

    file = open(RESULTS_FILE_NAME, "w+")
    file.write(baseline_name + ", " + loc_or_emo + ":\n\n")
    for key, val in best_params.items():
        file.write(key+ ": "+str(val)+"\n")
    file.write("\n")
    for i, res in enumerate(results):
        file.write(EVALUATION_RESULTS_KEYS[i]+str(res)+"\n")
    file.close()
    
def emo_vectorize_func(a, b):
    if b == 1:
        return a
    else:
        return 0
    
def vectorize_func(a, b):
    if a == b:
        return 1
    else:
        return 0

### DummyClassifier

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

#### 1. Emoji location prediction

In [13]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_loc_train)
dummy_loc_pred = dummy_clf.predict(X_test)
print(calc_scores(y_loc_test, dummy_loc_pred, multi_class=True))

[0.95331697 0.01254376 0.0125267  0.0125352 ]


#### 2. Emoji type prediction

In [14]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_emo_train)
dummy_emo_pred = dummy_clf.predict(X_test)
print(calc_scores(y_emo_test, dummy_emo_pred, multi_class=True))

[0.95275681 0.01218849 0.01216014 0.01217381]


### Linear SVM Baseline

Linear SVM is based on the liblinear library and is faster on large datasets. Not using dual optimization problem makes the training extremely fast (pseudoinverse). It is the fastest so both feature extraction methods (TF-IDF and word2vec) are used for comparison. TF-IDF works better, but adding word2vec to it (hybrid approach) adds a little bit of improvement.

In [15]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#### 1. Emoji location prediction

First, hyperparameter C is fine-tuned and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [16]:
# Parameter tuning

c_range = (-5, 15)
param_C = [2**i for i in range(c_range[0], c_range[1]+1)]

c_size = c_range[1]-c_range[0]+1
results_test = np.zeros(c_size, dtype = np.float64)
for i in range(c_size):
    C1 = param_C[i]
            
    svm_clf = LinearSVC(class_weight="balanced", dual=False, C=C1)
    results = kfold_score(svm_clf, X_train, y_loc_train, multi_class=False, should_print=False)
    results_test[i] = results[3]
    
print(results_test)
Ci = results_test.argmax()
C_best = param_C[Ci]

best_params = {}
best_params['C'] = C_best

best_estimator = LinearSVC(class_weight="balanced", dual=False, C=C_best)
best_estimator.fit(X_train, y_loc_train)
y_pred = best_estimator.predict(X_test)



for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_loc_test, y_pred))


# K-fold 

print("Emoji locations:\n")
print("TF-IDF results:")
results = do_test(best_estimator, X_tfidf, y_loc_tfidf, linear_svm_model_file_name, should_train_linear_svm)

# Writing results into a file
write_to_file(best_params, results, "linearSVM", "locations")

[0.23028489 0.23028376 0.23029767 0.23028452 0.23030461 0.23029545
 0.23028143 0.23029254 0.23028462 0.2302894  0.23030714 0.23028481
 0.23028517 0.23028792 0.23029287 0.23028203 0.23029724 0.23029529
 0.23029702 0.2302811  0.23026859]
C: 32
             precision    recall  f1-score   support

          0       0.96      0.58      0.72    540504
          1       0.14      0.74      0.23     48199

avg / total       0.89      0.59      0.68    588703

Emoji locations:

TF-IDF results:
Accuracy: 0.5936804401722894
Precision: 0.7419172790666412
Recall: 0.1362488768836736
F1: 0.23021844010072906
ratio of positive/negative predictions 0.44594263595817674


In [17]:
# Training and testing the best estimator

svm_clf = LinearSVC(class_weight="balanced", dual=False, C=0.05)
svm_clf.fit(X_train, y_loc_train)
y_loc_pred_svm = svm_clf.predict(X_test)
fold_scores = calc_scores(y_loc_pred_svm, y_loc_test, False)
print_scores(fold_scores)

Accuracy: 0.5943047682787416
Precision: 0.7397248905578954
Recall: 0.13611358196248802
F1: 0.2299204880345132
ratio of positive/negative predictions 0.44494932079503585


#### 2. Emoji type prediction

First, hyperparameter C is fine-tuned and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [18]:
# Parameter tuning

c_range = (-5, 15)
param_C = [2**i for i in range(c_range[0], c_range[1]+1)]

c_size = c_range[1]-c_range[0]+1
results_test = np.zeros(c_size, dtype = np.float64)
for i in range(c_size):
    C1 = param_C[i]
            
    svm_clf = LinearSVC(class_weight="balanced", dual=False, C=C1)
    results = kfold_score(svm_clf, X_train, y_loc_train, multi_class=True, should_print=False)
    results_test[i] = results[3]

print(results_test)
Ci = results_test.argmax()
C_best = param_C[Ci]

best_params = {}
best_params['C'] = C_best

best_estimator = LinearSVC(class_weight="balanced", dual=False, C=C_best)
best_estimator.fit(X_train, y_emo_train)
y_pred = best_estimator.predict(X_test)


for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_emo_test, y_pred))


# K-fold 

print("Emoji types:\n")
print("TF-IDF results:")
results = do_test(best_estimator, X_tfidf, y_emo_tfidf, linear_svm_model_file_name, should_train_linear_svm, True)

# Writing results into a file
write_to_file(best_params, results, "linearSVM", "emojis")

[0.01786508 0.01786551 0.01786535 0.01786492 0.01786473 0.01786443
 0.01786448 0.01786441 0.01786515 0.0178658  0.01786571 0.01786582
 0.01786452 0.01786481 0.01786438 0.01786558 0.01786561 0.01786529
 0.0178654  0.01786538 0.01786439]
C: 64
             precision    recall  f1-score   support

          0       0.93      0.98      0.96    540510
          1       0.00      0.00      0.00     10543
          2       0.00      0.00      0.00      4967
          3       0.00      0.00      0.00      5000
          4       0.00      0.00      0.00      2599
          5       0.00      0.00      0.00      2468
          6       0.00      0.00      0.00      2207
          7       0.00      0.00      0.00      1985
          8       0.00      0.00      0.00      1966
          9       0.00      0.00      0.00      1706
         10       0.00      0.00      0.00      1532
         11       0.00      0.00      0.00      1544
         12       0.01      0.00      0.01      1459
         13    

/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.9560166773259191
Precision: 0.020951892081461358
Recall: 0.013606945838380565
F1: 0.011532319107019644


In [19]:
# Training and testing the best estimator

svm_clf = LinearSVC(class_weight="balanced", dual=False, C=64)
svm_clf.fit(X_train, y_emo_train)
y_emo_pred_svm = svm_clf.predict(X_test)
fold_scores = calc_scores(y_emo_pred_svm, y_emo_test, True)
print_scores(fold_scores)

/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/marin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Accuracy: 0.9560279202958106
Precision: 0.021565301722035597
Recall: 0.010961516946732256
F1: 0.011549760583579058


#### 3. Merging emoji type prediction and emoji location prediction



In [ ]:
vfunc = np.vectorize(emo_vectorize_func)
y_emo_loc_pred_svm = vfunc(y_emo_pred_svm, y_loc_pred_svm)

print(f"y_pred for emojis:\n {y_emo_pred_svm[:400]}\n")
print(f"y_pred for locations:\n {y_loc_pred_svm[:400]}\n")
print(f"y_pred for emojis with locations:\n {y_emo_loc_pred_svm[:400]}\n")

fold_scores = calc_scores(y_emo_loc_pred_svm, y_emo_test, True)
print_scores(fold_scores)

### Bagging SVM

Warning! SVM is based on the libsvm library and it scales poorly with large datasets. That is why an ensemble (bagging) is used. Each classifier is trained on a portion of the data which greatly reduces training times and gives similar (if not better) results. Using 10 estimators it still takes a few hours to train. Results are just a bit better than a single linear SVM.

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

n_estimators = 10
bagging_svm_clf = BaggingClassifier(SVC(kernel='linear', class_weight='balanced'), max_samples=1.0 / n_estimators,
                                        n_estimators=n_estimators, bootstrap=False)


print("Emoji locations:\n")

do_test(bagging_svm_clf, X, y_l, bagging_model_file_name, should_train_bagging_svm)  

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

n_estimators = 10
bagging_svm_clf = BaggingClassifier(SVC(kernel='linear', class_weight='balanced'), max_samples=1.0 / n_estimators,
                                        n_estimators=n_estimators, bootstrap=False)

print("Emoji types:\n")

do_test(bagging_svm_clf, X, y_e, bagging_model_file_name, should_train_bagging_svm, True)

### Random Forest Baseline

Random forests are pretty fast and are generally better than the SVM. Higher accuracy and recall, but worse precision.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

#### 1. Emoji location prediction

First, hyperparameters min_samples_leaf and min_samples_split are fine-tuned using grid search and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [ ]:
random_forest_clf = RandomForestClassifier()

# Parameter tuning

param_grid = {
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}
scoring= 'f1'

# run grid search
clf = GridSearchCV(random_forest_clf, param_grid, cv=5, scoring=scoring)
clf.fit(X_train, y_loc_train)

# results
best_estimator = clf.best_estimator_
best_params = {}
best_params['min_samples_leaf'] = clf.best_params_['min_samples_leaf']
best_params['min_samples_split'] = clf.best_params_['min_samples_split']

y_pred = clf.predict(X_test)

for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_loc_test, y_pred))


# K-fold 

print("Emoji locations:\n")
results = do_test(best_estimator, X_tfidf, y_loc_tfidf, random_forest_file_name, should_train_random_forest)

# Writing results into a file
write_to_file(best_params, results, "randomForest", "locations")

In [ ]:
# Training and testing the best estimator

random_forest_clf = RandomForestClassifier(min_samples_leaf=1, min_samples_split=5)
random_forest_clf.fit(X_train, y_loc_train)

y_loc_pred_rf = random_forest_clf.predict(X_test)
fold_scores = calc_scores(y_loc_pred_rf, y_loc_test, False)
print_scores(fold_scores)

#### 2. Emoji type prediction

First, hyperparameters min_samples_leaf and min_samples_split are fine-tuned using grid search and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [ ]:
random_forest_clf = RandomForestClassifier()

# Parameter tuning

param_grid = {
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}
scoring= 'f1_micro'

# run grid search
clf = GridSearchCV(random_forest_clf, param_grid, cv=5, scoring=scoring)
clf.fit(X_train, y_emo_train)

# results
best_estimator = clf.best_estimator_
best_params = {}
best_params['min_samples_leaf'] = clf.best_params_['min_samples_leaf']
best_params['min_samples_split'] = clf.best_params_['min_samples_split']

y_pred = clf.predict(X_test)

for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_emo_test, y_pred))


# K-fold 

print("Emoji types:\n")
results = do_test(best_estimator, X_tfidf, y_emo_tfidf, random_forest_file_name, should_train_random_forest, True)

# Writing results into a file
write_to_file(best_params, results, "randomForest", "emojis")

In [ ]:
# Training and testing the best estimator
random_forest_clf = RandomForestClassifier(min_samples_leaf=4, min_samples_split=10)
random_forest_clf.fit(X_train, y_emo_train)

y_emo_pred_rf = random_forest_clf.predict(X_test)
fold_scores = calc_scores(y_emo_pred_rf, y_emo_test, True)
print_scores(fold_scores)

#### 3. Merging emoji type prediction and emoji location prediction

In [ ]:
vfunc = np.vectorize(emo_vectorize_func)
y_emo_loc_pred_rf = vfunc(y_emo_pred_rf, y_loc_pred_rf)

print(f"y_pred for emojis:\n {y_emo_pred_rf[:400]}\n")
print(f"y_pred for locations:\n {y_loc_pred_rf[:400]}\n")
print(f"y_pred for emojis with locations:\n {y_emo_loc_pred_rf[:400]}\n")

fold_scores = calc_scores(y_emo_loc_pred_rf, y_emo_test, True)
print_scores(fold_scores)

### AdaBoost Baseline

AdaBoost gives the best results and is relatively fast to train. It is slower than random forests, but still faster than bagging svms.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

#### 1. Emoji location prediction

First, hyperparameters base_estimator_max_depth, base_estimator_max_features and n_estimators are fine-tuned using grid search and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [ ]:
dtree_clf = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)
adaboost_clf = AdaBoostClassifier(base_estimator = dtree_clf)

# Parameter tuning

param_grid = {'base_estimator__max_depth': [2, 3, 4],
              'base_estimator__max_features': [2, 3, 4],
              'n_estimators': [50, 75]}
scoring = 'f1'

# run grid search
clf = GridSearchCV(adaboost_clf, param_grid, cv=5, scoring=scoring)
clf.fit(X_train, y_loc_train)

# results
best_estimator = clf.best_estimator_

best_params = {}
best_params['base_estimator__max_depth'] = clf.best_params_['base_estimator__max_depth']
best_params['base_estimator__max_features'] = clf.best_params_['base_estimator__max_features']
best_params['n_estimators'] = clf.best_params_['n_estimators']

y_pred = clf.predict(X_test)

for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_loc_test, y_pred))


# K-fold 

print("Emoji locations:\n")
results = do_test(best_estimator, X_tfidf, y_loc_tfidf, adaboost_file_name, should_train_adaboost)

# Writing results into a file
write_to_file(best_params, results, "adaBoost", "locations")

In [ ]:
# Training and testing the best estimator

dtree_clf = DecisionTreeClassifier(max_depth = 4, max_features = 4, random_state = 11, class_weight = "balanced")
adaboost_clf = AdaBoostClassifier(base_estimator = dtree_clf, n_estimators = 75)

adaboost_clf.fit(X_train, y_loc_train)

y_loc_pred_ab = adaboost_clf.predict(X_test)
fold_scores = calc_scores(y_loc_pred_ab, y_loc_test, False)
print_scores(fold_scores, False)

#### 2. Emoji type prediction

First, hyperparameters base_estimator_max_depth, base_estimator_max_features and n_estimators are fine-tuned using grid search and then results for the best estimator are calculated and written into a file. 
<br>
Second, the best estimator is trained and tested.

In [ ]:
dtree_clf = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)
adaboost_clf = AdaBoostClassifier(base_estimator = dtree_clf)

# Parameter tuning

param_grid = {'base_estimator__max_depth': [2, 3, 4],
              'base_estimator__max_features': [2, 3, 4],
              'n_estimators': [50, 75]}
scoring = 'f1_micro'

# run grid search
clf = GridSearchCV(adaboost_clf, param_grid, cv=5, scoring=scoring)
clf.fit(X_train, y_emo_train)

# results
best_estimator = clf.best_estimator_

best_params = {}
best_params['base_estimator__max_depth'] = clf.best_params_['base_estimator__max_depth']
best_params['base_estimator__max_features'] = clf.best_params_['base_estimator__max_features']
best_params['n_estimators'] = clf.best_params_['n_estimators']

y_pred = clf.predict(X_test)

for key, val in best_params.items():
    print(key+ ": "+str(val))
print(classification_report(y_emo_test, y_pred))


# K-fold 

print("Emoji locations:\n")
results = do_test(best_estimator, X_tfidf, y_emo_tfidf, adaboost_file_name, should_train_adaboost, True)

# Writing results into a file
write_to_file(best_params, results, "adaBoost", "emojis")

In [ ]:
# Training and testing the best estimator

dtree_clf = DecisionTreeClassifier(max_depth = 2, max_features = 2, random_state = 11, class_weight = "balanced")
adaboost_clf = AdaBoostClassifier(base_estimator = dtree_clf, n_estimators = 50)

adaboost_clf.fit(X_train, y_emo_train)

y_emo_pred_ab = adaboost_clf.predict(X_test)
fold_scores = calc_scores(y_emo_pred_ab, y_emo_test, True)
print_scores(fold_scores, True)

#### 3. Merging emoji type prediction and emoji location prediction

In [ ]:
vfunc = np.vectorize(emo_vectorize_func)
y_emo_loc_pred_ab = vfunc(y_emo_pred_ab, y_loc_pred_ab)

print(f"y_pred for emojis:\n {y_emo_pred_ab[:400]}\n")
print(f"y_pred for locations:\n {y_loc_pred_ab[:400]}\n")
print(f"y_pred for emojis with locations:\n {y_emo_loc_pred_ab[:400]}\n")

fold_scores = calc_scores(y_emo_loc_pred_ab, y_emo_test, True)
print_scores(fold_scores, True)

### T-test

In [ ]:
from scipy import stats

def test_vectorize_func(a, v1, v2):
    if a == 1:
        return v2
    else:
        return v1

vfunc = np.vectorize(test_vectorize_func)


#vec1 = 
#vec2 = 
#y_test = 

average = 'micro'  # for multi_class
#average = 'binary'

metric1 = []
metric2 = []

for i in range(0, 1000):
    vec_rand = np.random.randint(2, size=len(vec1))
    vec1_new = vfunc(vec_rand, vec1, vec2)
    vec2_new = vfunc(vec_rand, vec2, vec1)
    
    f1_vec1 = f1_score(y_test, vec1_new, average=average) 
    f1_vec2 = f1_score(y_test, vec2_new, average=average) 
    
    metric1.append(f1_vec1)
    metric2.append(f1_vec2)
    
print(stats.ttest_ind(metric1, metric2))

### BLSTM for emoji location prediction

Bidirectional long short-term memory recurrent network implementation using the Keras framework. Emoji location prediction is treated as a binary classification problem. TWEET_NUM determines how much of the tweets are used for training. The model uses an embedding layer and its size is a hyperparameter. Using pretrained Glove embeddings proved to be better. A single bidirectional layer is used which actually consists of two LSTM layers.

In [ ]:
from pandas import DataFrame

GLOVE_DIR = "./embeddings/"
GLOVE_FILE_NAME = "glove.twitter.27B."
GLOVE_FILE_NAME_EXT = "d.txt"

MODEL_DIR = "./models/blstm_models/"
BLSTM_BASE_FILE_NAME = "blstm_model_"
BLSTM_FILE_NAME_EXT = ".h5"

N_TIMESTEPS = MAX_WORDS_PER_TWEET + 1
NUM_EMOJI_TYPES = 20
TEST_SPLIT_SIZE = 0.1
VALIDATION_SPLIT = 0.1
EARLY_STOPPING_PATIENCE = 1
MAX_EPOCH = 30

TWEET_NUM = len(text_lines)
# TWEET_NUM = 100000
INPUT_SIZE = int(TWEET_NUM * (1 - TEST_SPLIT_SIZE))
NUM_OF_VOCAB = None
EMBEDDING_SIZE = 200
HIDDEN_SIZE = 1000

### Data preparation

Words are converted to integer ids using a tokenizer. Each unique word has its own unique integer value. Tweets are padded with zeros to a set length. Padding is required by Keras. Since most of the labels are zero (emojis are present after every 15th word), class weights are calculated. Without class weights the model behaves like a majority class classifier which is of no use here.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split

X_input = text_lines[:TWEET_NUM]
y_input = loc_lines[:TWEET_NUM]
y_emo_input = emo_lines[:TWEET_NUM]

abc = 100000

# X_blstm_train, y_blstm_train, y_blstm_emo_train = X_input[:abc], y_input[:abc], y_emo_input[:abc]
X_blstm_train, y_blstm_train, y_blstm_emo_train = X_input[:INPUT_SIZE], y_input[:INPUT_SIZE], y_emo_input[:INPUT_SIZE]
X_blstm_test, y_blstm_test, y_blstm_emo_test = X_input[INPUT_SIZE:], y_input[INPUT_SIZE:], y_emo_input[INPUT_SIZE:]

# X_blstm_train, X_blstm_test, y_blstm_train, y_blstm_test = train_test_split(X_input, y_input, test_size=TEST_SPLIT_SIZE)

if NUM_OF_VOCAB is not None:
    tokenizer = Tokenizer(num_words=NUM_OF_VOCAB)
else:
    tokenizer = Tokenizer()
    
tokenizer.fit_on_texts(X_input)
word_index = tokenizer.word_index
txt_to_seq = tokenizer.texts_to_sequences(X_blstm_train)
# print(f"encoded:\n{txt_to_seq[0:5]}\n")

if NUM_OF_VOCAB is not None:
    vocab_size = NUM_OF_VOCAB + 1
else:
    vocab_size = len(word_index) + 1

print(f"Vocabulary Size: {vocab_size}")

X_blstm = pad_sequences(txt_to_seq, maxlen=N_TIMESTEPS - 1, padding='post')
start_padding = np.zeros((X_blstm.shape[0], 1))
X_blstm = np.append(start_padding, X_blstm, axis=1).astype(int)
print(f"input shape: {X_blstm.shape}")
print(f"BLSTM input example:\n{X_blstm[:5]}\n")

y_loc = y_blstm_train
y_blstm = to_categorical(y_loc, num_classes=2)
print(f"location labels shape: {y_blstm.shape}\n")
print(f"BLSTM loc labels:\n{y_loc[:5]}\n")

y_emo = y_blstm_emo_train
y_emo_blstm = to_categorical(y_emo, num_classes=NUM_EMOJI_TYPES + 1)
print(f"emo labels shape: {y_emo_blstm.shape}\n")
print(f"BLSTM emo labels:\n{y_emo[:5]}\n")

def calc_sample_weights(y):
#     print(np.unique(y))
    weights = class_weight.compute_class_weight('balanced', np.unique(y), y.flatten())
    class_weight_dict = dict(enumerate(weights))
#     print(f"class weight dict:\n{class_weight_dict}\n")
    vfunc = np.vectorize(lambda x: class_weight_dict[x])
    return vfunc(y)

sample_weights_loc = calc_sample_weights(y_loc)
print(f"sample_weights_loc shape: {sample_weights_loc.shape}")
print(f"sample_weights_loc examples:\n{sample_weights_loc[:5]}\n")

sample_weights_emo = calc_sample_weights(y_emo)
print(f"sample_weights_emo shape: {sample_weights_emo.shape}")
print(f"sample_weights_emo examples:\n{sample_weights_emo[:5]}\n")

In [ ]:
import os

def load_embedding_matrix(glove_size):
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, GLOVE_FILE_NAME + str(glove_size) + GLOVE_FILE_NAME_EXT), encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((vocab_size, glove_size))
    for word, i in word_index.items():
        if i >= vocab_size - 1:
            break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i + 1] = embedding_vector

    print(f"embedding matrix shape {embedding_matrix.shape}")
    
    return embedding_matrix

glove_25 = None
glove_50 = None
glove_100 = None
glove_200 = None

def get_embedding_matrix(glove_size):
    global glove_25
    global glove_50
    global glove_100
    global glove_200
    
    if glove_size == 25:
        if glove_25 is None:
            glove_25 = load_embedding_matrix(25)
        return glove_25
    
    if glove_size == 50:
        if glove_50 is None:
            glove_50 = load_embedding_matrix(50)
        return glove_50
    
    if glove_size == 100:
        if glove_100 is None:
            glove_100 = load_embedding_matrix(100)
        return glove_100
    
    if glove_size == 200:
        if glove_200 is None:
            glove_200 = load_embedding_matrix(200)
        return glove_200

 ### Train model
 
 The following code is used to create and train a new model.

In [ ]:
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

def get_param_str(embedding_size, hidden_size, n_classes, input_size):
    param_str = "CLA-" + str(n_classes) + "_INP-" + str(input_size) + "_EMB-" + str(embedding_size) + "_HID-" + str(hidden_size)
    return param_str

def get_blstm_file_name(embedding_size, hidden_size, n_classes, input_size):
    param_str = get_param_str(embedding_size, hidden_size, n_classes, input_size)
    return MODEL_DIR + BLSTM_BASE_FILE_NAME + param_str + BLSTM_FILE_NAME_EXT

def save_blstm(blstm, embedding_size, hidden_size, n_classes=2, input_size=TWEET_NUM):
    blstm.save(get_blstm_file_name(embedding_size, hidden_size, n_classes, input_size))

def load_blstm(embedding_size, hidden_size, n_classes=2, input_size=TWEET_NUM):
    return load_model(get_blstm_file_name(embedding_size, hidden_size, n_classes, input_size))

def get_callbacks():
    callbacks = []
    
    callbacks.append(EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=EARLY_STOPPING_PATIENCE,
                              verbose=0, mode='auto'))
    checkpoint_path = MODEL_DIR + BLSTM_BASE_FILE_NAME + "check" + BLSTM_FILE_NAME_EXT
    callbacks.append(ModelCheckpoint(filepath=checkpoint_path, save_best_only=True))
    
    return callbacks

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dropout

def get_bi_lstm_model(embedding_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE, n_classes=2, use_glove=True,
                      n_timesteps=N_TIMESTEPS, mode="concat"):
    model = Sequential()
    
    if use_glove:
        embedding_matrix = get_embedding_matrix(embedding_size)
        model.add(Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=N_TIMESTEPS,
                            trainable=True))
    else:
        model.add(Embedding(vocab_size_size,
                            embedding_size,
                            input_length=N_TIMESTEPS))
    
    model.add(Dropout(0.5))
    
    model.add(Bidirectional(LSTM(hidden_size, return_sequences=True), merge_mode=mode))
    model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
#     binary_crossentropy
#     categorical_crossentropy
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"], sample_weight_mode="temporal")
    return model

def get_predictions(blstm, texts):
    text_to_integer_sequences = tokenizer.texts_to_sequences(texts)
    blstm_input = pad_sequences(text_to_integer_sequences, maxlen=N_TIMESTEPS, padding='post')
    ypred = blstm.predict_classes(blstm_input)
    return ypred

def evaluate_blstm(blstm, texts=X_blstm_test, labels=y_blstm_test):
    predictions = get_predictions(blstm, texts)
    print(predictions.shape)
    print(labels.shape)
    print(predictions[:10])
    print(labels[:10])
    print_scores(calc_scores(predictions.flatten(), labels.flatten(), np.max(labels)>1))

def do_blstm_test(embedding_size, hidden_size, is_emo=False, use_glove=True, input_size=TWEET_NUM):
    global blstm
    
    if is_emo:
        y_train = y_emo_blstm
        y_test = y_blstm_emo_test
        sample_weights = sample_weights_emo
    else:
        y_train = y_blstm
        y_test = y_blstm_test
        sample_weights = sample_weights_loc
        
    n_classes = y_train.shape[2]
    blstm = get_bi_lstm_model(embedding_size, hidden_size, n_classes, use_glove=use_glove)
    print(f"\nTraining with embedding {embedding_size} hidden {hidden_size} classes {n_classes} input {input_size} glove {use_glove}")    
    blstm.fit(X_blstm, y_train, epochs=MAX_EPOCH,
              validation_split=VALIDATION_SPLIT, verbose=2,
              sample_weight=sample_weights, callbacks=get_callbacks())
    save_blstm(blstm, embedding_size, hidden_size, n_classes, input_size)
    try:
        evaluate_blstm(blstm, labels=y_test)
    except Exception:
        print("couldn't evaluate")

In [ ]:
do_blstm_test(200, 500, is_emo=True)

In [ ]:
embedding_sizes = [200, 100]
hidden_sizes = [200, 500, 1000]

blstm = None
for embedding_size in embedding_sizes:
    for hidden_size in hidden_sizes:
        do_blstm_test(embedding_size, hidden_size, is_emo=False)
        
EARLY_STOPPING_PATIENCE = 30

do_blstm_test(200, 501, is_emo=False)

EARLY_STOPPING_PATIENCE = 1

do_blstm_test(200, 500, is_emo=True)

In [ ]:
# do_blstm_test(200, 500, is_emo=True)

blstm = load_blstm(200, 1000, 2)
blstm_emo = load_blstm(200, 500, 21)
imtired = 20
inputs = X_blstm_test[:imtired]
for i in range(imtired):
    print(inputs[i])
labels = y_blstm_test[:imtired]
labels_emo = y_blstm_emo_test[:imtired]

evaluate_blstm(blstm, inputs, labels)
evaluate_blstm(blstm_emo, inputs, labels_emo)

In [ ]:
do_blstm_test(200, 500, is_emo=True)

In [ ]:
evaluate_blstm(blstm, labels=y_blstm_test)

In [ ]:
save_blstm(blstm, EMBEDDING_SIZE, HIDDEN_SIZE)

In [ ]:
for embedding_size in embedding_sizes:
    for hidden_size in hidden_sizes:
        blstm = load_blstm(embedding_size, hidden_size, 2, 100000)
        evaluate_blstm(blstm, labels=y_blstm_test)

In [ ]:
for embedding_size in embedding_sizes:
    for hidden_size in hidden_sizes:
        blstm = load_blstm(embedding_size, hidden_size)
        print(f"\nEmbedding {embedding_size} hidden {hidden_size}")
        evaluate_blstm(blstm, X_blstm_test, y_blstm_test)
        print(f"test:")
        evaluate_blstm(blstm, X_blstm_test, y_blstm_test)
        print(f"\ntrain:")
        evaluate_blstm(blstm, X_blstm_train, y_blstm_train)

### Load a model

The following block is used to load models.

In [ ]:
blstm = load_model(BLSTM_MODEL_FILE_NAME)

In [ ]:
NUM_OF_TWEETS_TO_TEST = 10

print(f"example of labels (emoji locations):\n{y_input[:NUM_OF_TWEETS_TO_TEST]}\n")
print(f"model predictions:\n{get_predictions(blstm, X_input[:NUM_OF_TWEETS_TO_TEST])}\n")

### Load/Save predictions

Use the code below to save/load a model's predictions. Training and testing the model requires a powerful Nvidia GPU and is time consuming. This also enables the evaluation of results without having to work with the model itself.

In [ ]:
blstm_predictions_flag = False

if blstm_predictions_flag:
    BLSTM_PREDICTIONS_FILE_NAME = "blstm_predictions_" + PARAMETER_STR + ".pkl"
    predictions = get_predictions(text_lines)

    result_dict = {}
    result_dict["num_of_classes"] = N_CLASSES
    result_dict["num_of_input_tweets"] = TWEET_NUM
    result_dict["embedding_size"] = EMBEDDING_SIZE
    result_dict["hidden_size"] = HIDDEN_SIZE
    result_dict["input_X"] = X_blstm
    result_dict["input_y"] = y_blstm
    result_dict["ypred"] = predictions

    joblib.dump(result_dict, BLSTM_PREDICTIONS_FILE_NAME)

In [ ]:
if blstm_predictions_flag:
    result_dict = joblib.load(BLSTM_PREDICTIONS_FILE_NAME)